In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
import lxml
from lxml.etree import ElementTree,Element,SubElement
from lxml import etree
import lxml
from lxml import etree
import requests
from controle_coherence.utils import element_to_value_dict
mdd_path = Path("..")/"modele_donnee"

parser = etree.XMLParser(remove_blank_text=True)

#f = str((mdd_path/'exemples_metier'/'toto.xml'))
f = str((mdd_path/'exemples_metier'/'cas_test_immeuble_1_valid.xml'))

dpe = etree.parse(f, parser)

#for el in dpe.iterfind('*//adresse_brut'):
#    el.text=''
#    el.getparent().find('enum_statut_geocodage_ban_id').text="2"
#    adresse_dict = element_to_value_dict(el.getparent())

r=requests.post('http://localhost:5000/traduction_xml',data=etree.tostring(dpe))


In [6]:
r.content.decode()

'<dpe version="4.2.0">\n  <administratif>\n    <date_visite_diagnostiqueur>2021-07-10</date_visite_diagnostiqueur>\n    <date_etablissement_dpe>2021-07-10</date_etablissement_dpe>\n    <nom_proprietaire>Hubert Bonisseur de La Bath</nom_proprietaire>\n    <nom_proprietaire_installation_commune>Syndic Saint D\'hic</nom_proprietaire_installation_commune>\n    <enum_modele_dpe_id>DPE 3CL 2020 méthode logement</enum_modele_dpe_id>\n    <enum_version_id>2.1</enum_version_id>\n    <diagnostiqueur>\n      <usr_logiciel_id>3</usr_logiciel_id>\n      <version_logiciel>3.2</version_logiciel>\n      <nom_diagnostiqueur>Lamar</nom_diagnostiqueur>\n      <prenom_diagnostiqueur>Kendrick</prenom_diagnostiqueur>\n      <mail_diagnostiqueur>kendrick.lamar@diagnostiqueur.fr</mail_diagnostiqueur>\n      <telephone_diagnostiqueur>0607080910</telephone_diagnostiqueur>\n      <adresse_diagnostiqueur>20 avenue de Ségur 75007 Paris</adresse_diagnostiqueur>\n      <entreprise_diagnostiqueur>Entreprise Tartampio

In [7]:
dpe.find('*//date_etablissement_dpe').text="2300-01-01"

In [8]:
r=requests.post('http://localhost:5001/controle_coherence',data=etree.tostring(dpe))


In [9]:
r.json()

{'erreur_logiciel': [{'importance': 'bloquant',
   'message': "le DPE est fourni avec un numéro de version obsolète : 2.\n        date d'expiration de la version : 2200-01-01 00:00:00\n        Votre logiciel DPE doit être mis à jour. \n        les versions suivantes sont acceptées pour dépôt. ['1', '1.1']\n        ",
   'objets_concerne': ['/dpe/administratif/enum_version_id',
    '/dpe/administratif/date_etablissement_dpe'],
   'thematique': "la version de DPE n'est pas valable actuellement."}],
 'erreur_saisie': [{'importance': 'bloquant',
   'message': "le DPE est fourni avec une date d'établissement du DPE supérieure à la date d'aujourd'hui.\n            date d'établissement du DPE déclarée : 2300-01-01 00:00:00\n            date du jour : 2021-10-21\n\n            ",
   'objets_concerne': ['/dpe/administratif/enum_version_id',
    '/dpe/administratif/date_etablissement_dpe'],
   'thematique': "la date d'etablissement du DPE est supérieure à la date du jour et donc invalide."}],
 '

In [4]:
el_adresse.find('adresse_brut').text

In [5]:
el_adresse.find("adresse_brut").text+''

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [3]:
el_adresse.find('adresse_brut').text is not None

True

In [10]:
adresse_dict

{'adresse_brut': '',
 'code_postal_brut': 93130,
 'nom_commune_brut': 'Noisy-le-Sec',
 'label_brut': '48 Avenue Marceau 93130 Noisy-le-Sec',
 'enum_statut_geocodage_ban_id': 2,
 'ban_date_appel': '2021-07-10',
 'ban_id': 93053612000048,
 'ban_label': '48 Avenue Marceau 93130 Noisy-le-Sec',
 'ban_housenumber': 48,
 'ban_street': 'Avenue Marceau',
 'ban_citycode': 93053,
 'ban_postcode': 93130,
 'ban_city': 'Noisy-le-Sec',
 'ban_type': 'housenumber',
 'ban_score': 0.9741772727272727,
 'ban_x': 660505.01,
 'ban_y': 6865744.45}

In [11]:
r.content.decode()

'ERREUR INTERNE DU MOTEUR DE CONTROLE DE COHERENCE\nTraceback (most recent call last):\n  File "D:/Cstbdata/MONWORKSPACE/Gitlab/observatoire-dpe/svc_controle_coherence/app.py", line 66, in controle_coherence\n    resp = procedure_validation(xml_tree)\n  File "D:\\Cstbdata\\MONWORKSPACE\\Gitlab\\observatoire-dpe\\svc_controle_coherence\\controle_coherence\\validation.py", line 16, in procedure_validation\n    return engine.run_controle_coherence(dpe)\n  File "D:\\Cstbdata\\MONWORKSPACE\\Gitlab\\observatoire-dpe\\svc_controle_coherence\\controle_coherence\\controle_coherence.py", line 181, in run_controle_coherence\n    self.controle_coherence_administratif(dpe, report)\n  File "D:\\Cstbdata\\MONWORKSPACE\\Gitlab\\observatoire-dpe\\svc_controle_coherence\\controle_coherence\\controle_coherence.py", line 237, in controle_coherence_administratif\n    adresse_brut_concat = adresse_dict[\'adresse_brut\'] + \' \' + code_postal_brut + \' \' + adresse_dict[\'nom_commune_brut\']\nKeyError: \'adr

In [2]:
dpe

In [6]:
url = "http://localhost:5000/controle_coherence"
for cas_test_v1 in( mdd_path / 'exemples_editeurs').iterdir():
    dpe = etree.parse(str(cas_test_v1.absolute()), parser)
    r=requests.post(url,data=etree.tostring(dpe))
    resp = r.json()
    nb_errors = len(resp['erreur_logiciel']) + len(resp['erreur_saisie'])

    assert (r.status_code == 200)
    assert(nb_errors==0)
    assert(resp['validation_xsd']['valid']==True)



In [6]:
print(r.content.decode())

ERREUR INTERNE DU MOTEUR DE CONTROLE DE COHERENCE
Traceback (most recent call last):
  File "D:/Cstbdata/MONWORKSPACE/Gitlab/observatoire-dpe/svc_controle_coherence/app.py", line 66, in controle_coherence
    resp = procedure_validation(xml_tree)
  File "D:\Cstbdata\MONWORKSPACE\Gitlab\observatoire-dpe\svc_controle_coherence\controle_coherence\validation.py", line 16, in procedure_validation
    return engine.run_controle_coherence(dpe)
  File "D:\Cstbdata\MONWORKSPACE\Gitlab\observatoire-dpe\svc_controle_coherence\controle_coherence\controle_coherence.py", line 181, in run_controle_coherence
    self.controle_coherence_administratif(dpe, report)
  File "D:\Cstbdata\MONWORKSPACE\Gitlab\observatoire-dpe\svc_controle_coherence\controle_coherence\controle_coherence.py", line 259, in controle_coherence_administratif
    msg += resp_geocode
UnboundLocalError: local variable 'resp_geocode' referenced before assignment

        


In [2]:
adresse_brut_concat='de la Pinède 06160 JUAN-LES-PINS'

In [3]:
r = requests.get(f'https://api-adresse.data.gouv.fr/search/?q={adresse_brut_concat}')

In [4]:
r.json()

{'type': 'FeatureCollection',
 'version': 'draft',
 'features': [],
 'attribution': 'BAN',
 'licence': 'ETALAB-2.0',
 'query': 'de la Pinède 06160 JUAN-LES-PINS',
 'limit': 5}

In [33]:
r=requests.post('http://localhost:5000/controle_coherence',data='<badxml></badxml>')

In [34]:
r=requests.post('http://localhost:5000/controle_coherence',data='bad_content')
assert (r.status_code == 400)

In [11]:
r.json()

{'erreur_logiciel': [],
 'erreur_saisie': [],
 'message_principal': 'ERREUR VALIDATION XML : CONTROLE COHERENCE NON EFFECTUE.',
 'notification': [],
 'validation_xsd': {'error_log': "\nERREUR VALIDATION XML : l'élément /administratif/enum_version_id est manquant le processus de validation du DPE ne peut être effectué\n        ",
  'valid': False},
 'warning_logiciel': [],
 'warning_saisie': []}

In [3]:
import hashlib

'279fbea5c7970f4438df40702df3bb61'

In [2]:
r.json()

{'erreur_logiciel': [],
 'erreur_saisie': [],
 'message_principal': 'CONTROLE COHERENCE DU MODELE NON EFFECTUE SUR DPE LOGEMENT EXISTANT ALLEGE.\npour tester le controle de cohérence de la version complète,  enum_version_id doit être supérieur à 1.\n',
 'notification': [],
 'validation_xsd': {'error_log': [''], 'valid': True},
 'warning_logiciel': [],
 'warning_saisie': []}

In [35]:
parser = etree.XMLParser(remove_blank_text=True)
f = str((mdd_path/'exemples_metier'/'cas_test_appt_1.xml'))
dpe = etree.parse(f, parser)
f = str((mdd_path / 'exemples_defaut' / 'DPE_logement.xml'))
dpe = etree.parse(f, parser)
dpe.find('*enum_version_id').text='2'

r=requests.post('http://localhost:5000/controle_coherence',data=etree.tostring(dpe))


In [36]:
r.json()

{'erreur_logiciel': [{'importance': 'bloquant',
   'message': "\nmauvaise correspondance entre la valeur tv_deltar_id:1\navec les données connexes suivantes :\n{'enum_type_fermeture_id': [2], 'type_fermeture': 'Jalousie accordéon, fermeture à lames orientables y compris les vénitiens extérieurs tout métal, volets battants ou persiennes avec ajours fixes', 'deltar': 0.08}\net l'énumérateur enum_type_fermeture_id:{1: 'abscence de fermeture pour la baie vitrée'}.\nLa valeur attendue de l'énumérateur enum_type_fermeture_id doit être une des suivantes:\n{2: 'Jalousie accordéon, fermeture à lames orientables y compris les vénitiens extérieurs tout métal, volets battants ou persiennes avec ajours fixes'}\n",
   'objets_concerne': ['/dpe/logement/enveloppe/baie_vitree_collection/baie_vitree/donnee_entree/enum_type_fermeture_id',
    '/dpe/logement/enveloppe/baie_vitree_collection/baie_vitree/donnee_entree/tv_deltar_id'],
   'thematique': 'mauvaise correspondance enum avec table de valeur'},
  

In [25]:
r.json()['warning_saisie']

[]

In [22]:
for error in r.json()['erreur_logiciel']:
    print(error['message'])


mauvaise correspondance entre la valeur tv_deltar_id:1
avec les données connexes suivantes :
{'enum_type_fermeture_id': [2], 'type_fermeture': 'Jalousie accordéon, fermeture à lames orientables y compris les vénitiens extérieurs tout métal, volets battants ou persiennes avec ajours fixes', 'deltar': 0.08}
et l'énumérateur enum_type_fermeture_id:{1: 'abscence de fermeture pour la baie vitrée'}.
La valeur attendue de l'énumérateur enum_type_fermeture_id doit être une des suivantes:
{2: 'Jalousie accordéon, fermeture à lames orientables y compris les vénitiens extérieurs tout métal, volets battants ou persiennes avec ajours fixes'}


mauvaise correspondance entre la valeur tv_generateur_combustion_id:1
avec les données connexes suivantes :
{'enum_type_generateur_ch_id': [85, 127, 144], 'enum_type_generateur_ecs_id': [45, 92], 'type_generateur': 'Chaudière gaz  classique avant 1981', 'pn': 'Pn', 'rpn': '84 + 2 logPn', 'rpint': '80 + 3 logPn', 'qp0_perc': 0.04, 'pveil': 240.0}
et l'énuméra

In [23]:
for warning in r.json()['warning_saisie']:
    print(warning['message'])


Le resultat du géocodage n'est pas précis au numéro de rue mais à une échelle plus imprécise street
Ceci est anormal dans la plupart des cas. Essayez de préciser l'adresse au niveau du numéro de rue si celui-ci est bien référencé dans la Base d'Adresse Nationale
vous pouvez vous aider de cette carte centrée autour de l'adresse renseignée :  https://adresse.data.gouv.fr/base-adresse-nationale/String  


Le score de géocodage de l'adresse adresse_bien est faible : 0
Est ce que l'adresse géocodée : String correspond bien à l'adresse adresse_bien
                    

L'adresse adresse_proprietaire n'as pas été géocodée. Vérifier que l'adresse saisie String 00000 String est bien valide.
Il est possible que l'adresse adresse_proprietaire ne soit pas référencé dans la Base d'Adresse Nationale.
vous pouvez le vérfier grâce à cette carte interactive https://adresse.data.gouv.fr/base-adresse-nationale       
si votre adresse existe bien dans la Base d'Adresse Nationale, tentez de relancer le g